## Answer single Questions

In [27]:
import distilbert
from utils import *

testmodel = distilbert.DistilBERT()

Didn't find file ./test-squad-trained_distilbert\added_tokens.json. We won't load it.
loading file ./test-squad-trained_distilbert\vocab.txt
loading file ./test-squad-trained_distilbert\tokenizer.json
loading file None
loading file ./test-squad-trained_distilbert\special_tokens_map.json
loading file ./test-squad-trained_distilbert\tokenizer_config.json
loading configuration file ./test-squad-trained_distilbert\config.json
Model config DistilBertConfig {
  "_name_or_path": "./test-squad-trained_distilbert",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",

In [2]:
testmodel.predict("who am I?", "My name is Roman.")

{'score': 0.7300494909286499, 'start': 11, 'end': 16, 'answer': 'Roman'}

In [3]:
testmodel.predict("who am I?", "My name is Roman but not Vincenzo.")

{'score': 0.41747188568115234, 'start': 11, 'end': 16, 'answer': 'Roman'}

In [4]:
testmodel.predict("who am I?", "My name is Roman and Vincenzo.")

{'score': 0.3489514887332916,
 'start': 11,
 'end': 29,
 'answer': 'Roman and Vincenzo'}

In [5]:
testmodel.predict("who am I?", "My name is not Roman and but Vincenzo.")

{'score': 0.2312117964029312, 'start': 29, 'end': 37, 'answer': 'Vincenzo'}

## Answer multiple Questions

In [6]:
from transformers import Trainer, default_data_collator, AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments
from datasets import load_dataset, load_metric
import torch
import numpy as np

In [7]:
tokenizer = AutoTokenizer.from_pretrained("./test-squad-trained_distilbert")
model = AutoModelForQuestionAnswering.from_pretrained("./test-squad-trained_distilbert").to("cuda:0")

config = load_yaml_file('distilbert_config.yaml')

##
GPU_USAGE = config['GPU_USAGE']
BATCH_SIZE = config['BATCH_SIZE']
MAX_LENGTH = config['MAX_LENGTH']
DOC_STRIDE = config['DOC_STRIDE']
MAX_ANSWER_LENGTH = config['MAX_ANSWER_LENGTH']
squad_v2 = config['squad_v2']
N_BEST_SIZE = config['N_BEST_SIZE']
##
PAD_RIGHT = tokenizer.padding_side == "right"

datasets = load_dataset("squad_v2")
data_collator = default_data_collator

Reusing dataset squad_v2 (C:\Users\vince\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)
100%|██████████| 2/2 [00:00<00:00, 27.60it/s]


In [8]:
torch.cuda.is_available()

True

In [9]:
tokenized_datasets = datasets.map(
    prepare_train_features, 
    fn_kwargs={
        'tokenizer':tokenizer, 
        'PAD_RIGHT':PAD_RIGHT,
        'MAX_LENGTH':MAX_LENGTH, 
        'DOC_STRIDE':DOC_STRIDE
        }, 
    batched=True, remove_columns=datasets["train"].column_names)

Loading cached processed dataset at C:\Users\vince\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d\cache-baf0920fff84bf37.arrow
Loading cached processed dataset at C:\Users\vince\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d\cache-14ed320150076ffd.arrow


In [10]:
model_name = "test-squad-trained"

import os
os.environ["WANDB_DISABLED"] = "true"

args = TrainingArguments(
    f"{model_name}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,

)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [11]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    fn_kwargs={
        'tokenizer':tokenizer, 
        'PAD_RIGHT':PAD_RIGHT, 
        'MAX_LENGTH':MAX_LENGTH, 
        'DOC_STRIDE':DOC_STRIDE
        },
    batched=True,
    remove_columns=datasets["validation"].column_names
)

Loading cached processed dataset at C:\Users\vince\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d\cache-71e2e1c0522b5336.arrow


In [12]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `DistilBertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 12134
  Batch size = 16
100%|██████████| 759/759 [01:58<00:00,  6.47it/s]

In [13]:
for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [14]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -N_BEST_SIZE - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -N_BEST_SIZE - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
                start_index >= len(offset_mapping)
                or end_index >= len(offset_mapping)
                or offset_mapping[start_index] is None
                or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > MAX_ANSWER_LENGTH:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:N_BEST_SIZE  ]
valid_answers

[{'score': 14.623622, 'text': 'France'},
 {'score': 8.509556, 'text': 'France.'},
 {'score': 6.3958488, 'text': 'in France'},
 {'score': 5.27246, 'text': 'a region in France'},
 {'score': 4.360544, 'text': 'region in France'},
 {'score': 4.3407164, 'text': 'Normandy, a region in France'},
 {'score': 2.464447,
  'text': 'in the 10th and 11th centuries gave their name to Normandy, a region in France'},
 {'score': 2.3809881,
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark'},
 {'score': 2.0457087,
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway'},
 {'score': 1.5203052,
  'text': '10th and 11th centuries gave their name to Normandy, a region in France'},
 {'score': 1.1334071,
  'text': 'France. They were descended from Norse ("Norman" comes from "Norseman"'},
 {'score': 0.9327135,
  'text': 'France. They were descended from Norse ("Norma

In [15]:
final_predictions = postprocess_qa_predictions(
    datasets["validation"], 
    validation_features, 
    raw_predictions.predictions,
    tokenizer, 
    config['squad_v2'], 
    n_best_size=config['N_BEST_SIZE'], 
    max_answer_length=config['MAX_ANSWER_LENGTH'])

Post-processing 11873 example predictions split into 12134 features.


100%|██████████| 11873/11873 [00:23<00:00, 515.48it/s]


In [16]:
metric = load_metric("squad_v2" if squad_v2 else "squad")
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 59.56371599427272,
 'f1': 65.17491830846696,
 'total': 11873,
 'HasAns_exact': 55.66801619433198,
 'HasAns_f1': 66.90651232733147,
 'HasAns_total': 5928,
 'NoAns_exact': 63.44827586206897,
 'NoAns_f1': 63.44827586206897,
 'NoAns_total': 5945,
 'best_exact': 59.57213846542576,
 'best_exact_thresh': 0.0,
 'best_f1': 65.17491830846737,
 'best_f1_thresh': 0.0}

In [34]:
bleus = calculate_bleu_score_new(formatted_predictions, references)
bleu_avg = np.sum(bleus)/len(bleus)
print('BLEU-2 avg:', bleu_avg, 'of', len(bleus), 'comparisons')

BLEU-2 avg: 0.6435269107841263 of 11873 comparisons


In [33]:
bleus = calculate_bleu_score(formatted_predictions, references)
bleu_avg = np.sum(bleus)/len(bleus)
print('BLEU-2 avg:', bleu_avg, 'of', len(bleus), 'comparisons')

BLEU-2 avg: 0.7994616680595023 of 4839 comparisons


In [ ]:
tp, fp, tn, fn = [], [], [], []
for i in range(len(formatted_predictions)):
    pred = formatted_predictions[i]['prediction_text']
    ref = references[i]['answers']['text']
    if (len(pred) == 0) and (len(ref) == 0):
        tn.append(i)
    if (len(pred) != 0) and (len(ref) == 0):
        fp.append(i)
    if (len(pred) == 0) and (len(ref) != 0):
        fn.append(i)
    if (len(pred) != 0) and (len(ref) != 0):
        tp.append(i)
    
print('{} tp | {} fp\n-----------------\n{} fn | {} tn'.format(len(tp),len(fp),len(fn),len(tn)))

4838 tp | 2173 fp
-----------------
1090 fn | 3772 tn


In [ ]:
fp_n = 3
np.random.seed(42)
idxs_fp = np.random.choice(len(fp),fp_n,False)

for idx_fp in idxs_fp:
    idx_fp = fp[int(idx_fp)]
    print(str(idx_fp))
    print(str(datasets['validation'][idx_fp]))
    print('\n'+str(formatted_predictions[idx_fp]))
    print('\n'+str(references[idx_fp])+'\n\n\n')

1375
{'id': '5a637c6268151a001a922311', 'title': 'Victoria_(Australia)', 'context': 'In 1854 at Ballarat there was an armed rebellion against the government of Victoria by miners protesting against mining taxes (the "Eureka Stockade"). This was crushed by British troops, but the discontents prompted colonial authorities to reform the administration (particularly reducing the hated mining licence fees) and extend the franchise. Within a short time, the Imperial Parliament granted Victoria responsible government with the passage of the Colony of Victoria Act 1855. Some of the leaders of the Eureka rebellion went on to become members of the Victorian Parliament.', 'question': 'What did some leaders of the British rebellion become?', 'answers': {'text': [], 'answer_start': []}}

{'id': '5a637c6268151a001a922311', 'prediction_text': 'members of the Victorian Parliament', 'no_answer_probability': 0.0}

{'id': '5a637c6268151a001a922311', 'answers': {'text': [], 'answer_start': []}}



9220
{'

In [ ]:
fn_n = 3
np.random.seed(42)
idxs_fn = np.random.choice(len(fn),fn_n,False)

for idx_fn in idxs_fn:
    idx_fn = fn[int(idx_fn)]
    print(str(idx_fn))
    print(str(datasets['validation'][idx_fn]))
    print('\n'+str(formatted_predictions[idx_fn]))
    print('\n'+str(references[idx_fn])+'\n\n\n')

9252
{'id': '572ff56304bcaa1900d76f2d', 'title': 'Rhine', 'context': 'The other third of the water flows through the Pannerdens Kanaal and redistributes in the IJssel and Nederrijn. The IJssel branch carries one ninth of the water flow of the Rhine north into the IJsselmeer (a former bay), while the Nederrijn carries approximately two ninths of the flow west along a route parallel to the Waal. However, at Wijk bij Duurstede, the Nederrijn changes its name and becomes the Lek. It flows farther west, to rejoin the Noord River into the Nieuwe Maas and to the North Sea.', 'question': 'If two thirds of the Rhine flows through Waal, where does the other third flow through?', 'answers': {'text': ['Pannerdens Kanaal', 'Pannerdens Kanaal', 'the Pannerdens Kanaal'], 'answer_start': [47, 47, 43]}}

{'id': '572ff56304bcaa1900d76f2d', 'prediction_text': '', 'no_answer_probability': 0.0}

{'id': '572ff56304bcaa1900d76f2d', 'answers': {'text': ['Pannerdens Kanaal', 'Pannerdens Kanaal', 'the Pannerden

In [ ]:
from transformers import pipeline
model_cpu = AutoModelForQuestionAnswering.from_pretrained("./test-squad-trained_distilbert")
qa_prediction = pipeline('question-answering', model=model_cpu, tokenizer=tokenizer)

loading configuration file ./test-squad-trained_distilbert/config.json
Model config DistilBertConfig {
  "_name_or_path": "./test-squad-trained_distilbert",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "vocab_size": 30522
}

loading weights file ./test-squad-trained_distilbert/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForQuestionAnswering.

All the weights of DistilBertForQuestionAnswering were initialized from the model checkpoint at ./test-squad-trained_distilbert.
If your task is similar to the task the